In [1]:
import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
# import pyLDAvis
# import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_path = "/content/drive/MyDrive/capstone/Data/"
data = pd.read_csv(data_path+"Data_clean_processed_sample.csv")

## LDA

In [4]:
if type(data['bow']) is list:
    text = data['bow']
else:
    text = data['bow'].tolist()

text=[str(x).split() for x in text]

In [5]:
id2word = corpora.Dictionary(text)  
# Create Corpus 
texts = text  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  

In [6]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [7]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.011*"market" + 0.010*"power" + 0.008*"energy" + 0.008*"gas" + '
  '0.008*"company" + 0.007*"business" + 0.006*"attached" + 0.006*"service" + '
  '0.006*"said" + 0.005*"new"'),
 (1,
  '0.012*"updated" + 0.010*"week" + 0.008*"play" + 0.008*"game" + 0.007*"one" '
  '+ 0.007*"day" + 0.006*"texas" + 0.006*"season" + 0.006*"houston" + '
  '0.006*"year"'),
 (2,
  '0.015*"agreement" + 0.015*"date" + 0.013*"trade" + 0.012*"contract" + '
  '0.012*"day" + 0.012*"deal" + 0.011*"transaction" + 0.010*"please" + '
  '0.010*"request" + 0.010*"file"'),
 (3,
  '0.072*"enron" + 0.015*"mail" + 0.014*"company" + 0.012*"com" + 0.011*"may" '
  '+ 0.011*"employee" + 0.011*"said" + 0.010*"corp" + 0.009*"investment" + '
  '0.009*"message"'),
 (4,
  '0.035*"urlmasker" + 0.016*"com" + 0.011*"new" + 0.009*"click" + '
  '0.007*"venture" + 0.007*"free" + 0.007*"round" + 0.007*"service" + '
  '0.007*"million" + 0.007*"site"'),
 (5,
  '0.063*"imagemasker" + 0.020*"total" + 0.011*"provider" + 0.011*"desk" + '

## Topic evolution

In [15]:
probability = []
for i in doc_lda: 
  a = [0]*7
  for x in i[0]:
    a[x[0]] = x[1]
  probability.append(np.array(a))
  # print(probability)
  # break

In [ ]:
probs = np.array(probability)

### Get an email with threads

In [ ]:
data[['email','id']] = data['email_id'].str.split('#',1,expand = True)
df = data['email'].value_counts()
df = df.reset_index()
thread = data.merge(df,left_on = 'email',right_on = 'index')
thread = thread[thread['email_y']>1]
thread = thread.sort_values('email_y')
thread['id'] = thread['id'].map(int)

### Averaged topic probability

In [ ]:
email_id = thread['index'].iloc[1650]
threads = thread[thread['email_x'] == email_id].sort_values('id')
result = []
prob_origin = []
topic_evolv = []
topic_origin = []
for i in range(len(threads)):
  body = threads['bow'].iloc[i].split()
  weight = min(1,1/(np.log(len(body)+1)+1)*3)
  if i==0:
    topic_evolv.append(str(np.argmax(probs[threads.index[i]])))
    topic_origin.append(str(np.argmax(probs[threads.index[i]])))
    result.append(probs[threads.index[i]])
    prob_origin.append(probs[threads.index[i]])
  else:
    # print((weight)*result[-1])
    result.append((weight)*result[-1]+(1-weight)*probs[threads.index[i]])
    topic_evolv.append(str(np.argmax(result[-1])))
    topic_origin.append(str(np.argmax(probs[threads.index[i]])))
    prob_origin.append(probs[threads.index[i]])
                  
print(topic_evolv)
print(topic_origin)

In [ ]:
topic_word = ['6']
for i in lda_model.get_topic_terms(6,10):
  topic_word.append(id2word.id2token[i[0]])
'_'.join(topic_word)

'6_please_would_know_time_need_get_price_let_call_thanks'

In [ ]:
!pip install pyLDAvis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models 

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.199133  0.042331       1        1  25.807803
5     -0.185060 -0.159422       2        1  21.007657
0     -0.156091  0.179097       3        1  20.873803
1      0.102563 -0.098420       4        1  10.747209
3      0.107839 -0.195516       5        1   9.854428
2      0.160041  0.135768       6        1   6.218820
6      0.169841  0.096162       7        1   5.490281, topic_info=             Term         Freq        Total Category  logprob  loglift
2026    urlmasker  3972.000000  3972.000000  Default  30.0000  30.0000
52         please  6680.000000  6680.000000  Default  29.0000  29.0000
655   imagemasker  2140.000000  2140.000000  Default  28.0000  28.0000
1059        price  1890.000000  1890.000000  Default  27.0000  27.0000
223           get  3686.000000  3686.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
2862        total   477.199089   670.977882   Topic7  -4.9917   2.5614
737         quote   165.829131   216.802555   Topic7  -6.0487   2.6342
672           low   197.915703   633.910819   Topic7  -5.8718   1.7381
809          true   154.670141   227.484680   Topic7  -6.1184   2.5164
69           deal   160.370157  1980.639472   Topic7  -6.0822   0.3885

[369 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
54684      7  0.993353       $--
5102       7  0.997394       $..
882        1  0.999079  $million
390        5  0.999128       (or
9183       6  0.999021     -----
...      ...       ...       ...
4265       6  0.997485      yard
866        1  0.392528      year
866        2  0.029476      year
866        3  0.578009      year
3860       6  0.994512       yes

[480 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 6, 1, 2, 4, 3, 7])

In [ ]:
data_path = "/content/drive/MyDrive/capstone/Models Comparison/"

In [ ]:
file_name=["all-MiniLM-L6-v2",10,(1,2),7,10,50]
file_name=[str(x) for x in file_name]
file_name="_".join(file_name)

In [ ]:
bert_topics = pd.read_csv(data_path + r'BERTopic/top_words_{}_mask.csv'.format(file_name))
bert_assign = pd.read_csv(data_path + r'BERTopic/assignments_{}_mask.csv'.format(file_name))


In [ ]:
bert_assign['email'] = bert_assign['bow']

In [ ]:
lda_assign = pd.read_csv(data_path+"LDA4_emails_assignments.csv")
lda_topics = pd.read_csv(data_path+"LDA4_topics_100.csv")

In [ ]:
!pip install sentence_transformers

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
lda_topics = lda_topics.groupby('topic').head(10).reset_index()
lda_topics['embedding'] = [x for x in embedding_model.encode(lda_topics['term'])]
lda_topics = lda_topics.assign(beta_sum=lda_topics.groupby('topic')['beta'].transform('sum'))
lda_topics['topic_embedding'] = lda_topics.apply(lambda row: row['beta']*row['embedding']/row['beta_sum'], axis = 1)
lda_topic = lda_topics.groupby(['topic'])['topic_embedding'].sum().to_frame(name = 'embedding').reset_index()